In [193]:
import pandas as pd
from fecfile.dataframe import Fecfile

In [136]:
from pathlib import Path
paths = list(Path("cache").glob("*/*/*.parquet"))


In [137]:
import random
random.shuffle(paths)


In [138]:
big = [p for p in paths if p.stat().st_size > 2e5]
len(big)
big[:20]

[PosixPath('cache/20190325/SA11AI/1321100.parquet'),
 PosixPath('cache/20191015/SA11AI/1357152.parquet'),
 PosixPath('cache/20200131/SA17/1376657.parquet'),
 PosixPath('cache/20100115/SB23/445110.parquet'),
 PosixPath('cache/20190131/SA16/1312107.parquet'),
 PosixPath('cache/20191218/SA11AI/1366633.parquet'),
 PosixPath('cache/20190131/SA11AI/1312469.parquet'),
 PosixPath('cache/20190128/SA11AI/1307531.parquet'),
 PosixPath('cache/20190201/SA11AI/1312796.parquet'),
 PosixPath('cache/20190319/SA11AI/1320134.parquet'),
 PosixPath('cache/20191203/SB23/1365089.parquet'),
 PosixPath('cache/20191120/SA11AI/1364197.parquet'),
 PosixPath('cache/20190809/SA11AI/1347373.parquet'),
 PosixPath('cache/20191120/SA11AI/1364180.parquet'),
 PosixPath('cache/20190809/SA11AI/1347320.parquet'),
 PosixPath('cache/20190716/SA17A/1341083.parquet'),
 PosixPath('cache/20190719/SA11ai/1342289.parquet'),
 PosixPath('cache/20190731/SA11AI/1346218.parquet'),
 PosixPath('cache/20200320/SA11AI/1391220.parquet'),
 Po

In [161]:
fname = random.choice(big)

_, date, form, num = fname.with_suffix("").parts



'SA11AI'

In [202]:

def network_els(filing):
    # Break down a long filing parquet frame into just the basics; donor, recipient, date, and amount.
    # Donor identification is kind of dicey.
    frame = pd.read_parquet(filing)
    if not isinstance(frame.index, pd.RangeIndex):
        # A bugfix against a previous version--this code sucks.
        p = Path(filing)
        _, date, form, id = p.with_suffix("").parts
        redone = Fecfile(Path("fec") / date / (id + ".fec"))
        redone.prepare_itemization_buffers()
        reformed = redone.to_pandas(form)
        reformed.to_parquet(p)
        frame = reformed
    try:
        frame.loc[:,'zip'] = frame.contributor_zip_code.str.slice(0, 5)
        donor = frame[['contributor_first_name','contributor_last_name','contributor_organization_name','zip']]
        donor = donor.fillna("")
        donor = "I_" + donor.contributor_first_name + "_" + donor.contributor_last_name.replace("'", "") + "_" + donor.zip + "_" + donor.contributor_organization_name
        donor[pd.notna(frame.donor_committee_fec_id)] = frame.donor_committee_fec_id
        recipient = frame.filer_committee_id_number
        amount = frame.contribution_amount
        date = frame.contribution_date
    except AttributeError:
        if "beneficiary_candidate_fec_id" in frame.columns:
            donor = frame.filer_committee_id_number
            recipient = frame.beneficiary_candidate_fec_id
            amount = frame.expenditure_amount
            date = frame.expenditure_date
            recipient[pd.isna(recipient)] = frame.beneficiary_committee_fec_id[pd.isna(recipient)]
            recipient[pd.isna(recipient)] = frame.payee_organization_name[pd.isna(recipient)]
        else:
            return pd.DataFrame()
    result = pd.DataFrame({'date': date, 'recipient': recipient, 'amount': amount, "donor": donor.str.upper()})
    return result

In [188]:
isinstance(tots.index, pd.RangeIndex)

True

In [239]:
g = pd.read_parquet("cache/20200630/SA17/1415440.parquet")
f = network_els("cache/20200630/SA17/1415440.parquet")
f.amount.astype("<f4")

0    25000.0
1     7500.0
2     7500.0
3    45000.0
4    25000.0
Name: amount, dtype: float32

In [199]:
form

'F1A'

In [213]:
def summarize(date):
    short = Path(f"totals/{date}.parquet")
    individual = Path(f"totals/{date}_long.parquet")
    if short.exists():
        return
    fs = []
    for p in Path(f"cache/{date}").glob("*/*.parquet"):
        data = network_els(p)

        fs.append(data)
    tots = pd.concat(fs)
    tots.to_parquet(individual)
    tots = tots.groupby(["donor", "recipient"])['amount'].sum().reset_index()
    tots.to_parquet(short)

summarize("20200105")

/Users/bschmidt/miniconda3/envs/htrc/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/bschmidt/miniconda3/envs/htrc/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [228]:
dates = list(Path("fec").glob("*"))

for date in dates:
    for month in "345678":
        if date.name.startswith(f"20200{month}"):
            print(date)
            summarize(date.name)

fec/20200301
fec/20200302
fec/20200303


/Users/bschmidt/miniconda3/envs/htrc/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/bschmidt/miniconda3/envs/htrc/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


fec/20200304
fec/20200305
fec/20200306
fec/20200307
fec/20200308
fec/20200309
fec/20200310
fec/20200311
fec/20200312
fec/20200313
fec/20200314
fec/20200315
fec/20200316
fec/20200317
fec/20200318
fec/20200319
fec/20200320
fec/20200321
fec/20200322
fec/20200323
fec/20200324
fec/20200325
fec/20200326
fec/20200327
fec/20200328
fec/20200329
fec/20200330
fec/20200331
fec/20200401
fec/20200402
fec/20200403
fec/20200404
fec/20200405
fec/20200406
fec/20200407
fec/20200408
fec/20200409
fec/20200410
fec/20200411
fec/20200412
fec/20200413
fec/20200414
fec/20200415


ArrowInvalid: ('Could not convert 100.00 with type str: tried to convert to double', 'Conversion failed for column amount with type object')